# Calculate average precipitation rate of MCSs

In [1]:
import pathlib

import numpy as np
import xarray as xr

MCS by day

Steps:
1. Make blobs binary (1 or 0)
2. Take sum of binary blobs for each day
    1. For any days with sum > 3 (over 3 hours of MCS rain), set it as 1
    2. Otherwise, 0
3. Take sum of precipitation for each day
4. Multiply by MCS days from step 2
5. Sum and divide by 12 and multiply by (3600 * 1000), where 12 is number of years
6. To get MCS precip %, divide these results by the JJ avg prect (can be the monthly file multiplied by 61)

### Calculate binary blobs

### Group binary blobs by day:

In [15]:
data_dir = pathlib.Path("/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs")
files = sorted(data_dir.glob("era5_blobs_*.compressed.nc"))
files

[PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1991.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1992.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1993.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1994.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1995.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1996.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1997.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1998.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1999.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/n

In [16]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/"
for file in files:
    # Read in file
    print(f"Processing {file}")
    ds = (
        xr.open_dataset(file, engine="netcdf4")
        .sel(lat=slice(55, 5))
        .sel(lon=slice(235, 295))
    )
    # Convert hourly data to daily
    ds = ds.resample(time="D").sum()  # where D is for days
    # set days with MCS as 1, otherwise 0
    ds = xr.where(ds > 0, 1, 0)

    # save results
    name = file.stem.split("/")

    fname = (
        str(name[0]) + "_binary_days"
    )  # str(name[2]) + '_' + str(name[3]) + '_' + str(name[4])

    print("Exporting...")
    ds.to_netcdf(path + fname + ".nc")
    print("Saved!")

Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1991.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1992.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1993.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1994.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1995.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1996.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1997.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/era5_blobs_1998.compressed.nc
Exporting...
Saved!
Processing /glade/derech

### Group PRECT by day, then multiply with binary blob days

In [2]:
data_dir_blobs = pathlib.Path(
    "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs//"
)
b_files = sorted(data_dir_blobs.glob("era5_blobs_*.compressed_binary_days.nc"))
len(b_files)

11

In [3]:
p_path = pathlib.Path("/glade/derecho/scratch/malbright/prect_nam_mcs_hourly1/era5")

In [4]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/"
for i in range(len(b_files)):
    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")
    year = np.unique(ds_b.time.dt.year)[0]
    print(f"\nProcessing year {year}")
    print(ds_b.time)
    p_files = sorted(
        p_path.glob(f"e5.oper.fc.sfc.meanflux.235_055_mtpr.ll025sc*{year}*.nc")
    )
    ds_p = (
        xr.open_mfdataset(p_files)
        .sel(latitude=slice(55, 5))
        .sel(longitude=slice(235, 295))
        .sel(time=slice(f"{str(year)}-01-01", f"{str(year)}-12-31"))
        .load()
    )
    ds_p = ds_p.rename({'latitude': 'lat', 'longitude': 'lon'})
    # ds_p = ds_p.sel(time=is_month(ds_p['time.month']))
    # resample prect
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    print(ds_p.time)
    # Compute mcs rain rate
    ds = ds_b["MCS_MTPR"] * ds_p["MTPR"]

    # save results
    ds.to_netcdf(f"{path}mcs_day_prect_era5_{year}.nc")
    print("    Saved!")


Processing year 1991
<xarray.DataArray 'time' (time: 365)> Size: 3kB
array(['1991-01-01T00:00:00.000000000', '1991-01-02T00:00:00.000000000',
       '1991-01-03T00:00:00.000000000', ..., '1991-12-29T00:00:00.000000000',
       '1991-12-30T00:00:00.000000000', '1991-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 3kB 1991-01-01 1991-01-02 ... 1991-12-31
<xarray.DataArray 'time' (time: 365)> Size: 3kB
array(['1991-01-01T00:00:00.000000000', '1991-01-02T00:00:00.000000000',
       '1991-01-03T00:00:00.000000000', ..., '1991-12-29T00:00:00.000000000',
       '1991-12-30T00:00:00.000000000', '1991-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 3kB 1991-01-01 1991-01-02 ... 1991-12-31
    Saved!

Processing year 1992
<xarray.DataArray 'time' (time: 366)> Size: 3kB
array(['1992-01-01T00:00:00.000000000', '1992-01-02T00:00:00.000000000',
       '1992-01-03T00:00:00.000000000', 

### Calculate MCS precip %

In [7]:
data_dir_blobs = pathlib.Path(
    "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/"
)
b_files = sorted(data_dir_blobs.glob("mcs_day_prect_era5_*.nc"))
len(b_files)

11

In [9]:
p_path = pathlib.Path("/glade/derecho/scratch/malbright/prect_nam_mcs_hourly1/era5")

In [10]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/"
for i in range(len(b_files)):

    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")
    year = np.unique(ds_b.time.dt.year)[0]
    print(f"\nProcessing year {year}")

    ds_b = ds_b.sel(time=ds_b.time.dt.month.isin([3, 4, 5]))
    ds_b_total = ds_b.mean(dim='time') * len(ds_b.time)

    p_files = sorted(
        p_path.glob(f"e5.oper.fc.sfc.meanflux.235_055_mtpr.ll025sc*{year}*.nc")
    )
    ds_p = (
        xr.open_mfdataset(p_files)
        .sel(latitude=slice(55, 5))
        .sel(longitude=slice(235, 295))
        .sel(time=slice(f"{str(year)}-01-01", f"{str(year)}-12-31"))
        .load()
    )
    ds_p = ds_p.rename({'latitude': 'lat', 'longitude': 'lon'})

    ds_p = ds_p.sel(time=ds_p.time.dt.month.isin([3, 4, 5]))
    # resample prect
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    ds_p_total = ds_p.mean(dim='time') * len(ds_p.time)

    # Compute mcs rain rate %
    ds = ds_b_total["__xarray_dataarray_variable__"] / ds_p_total["MTPR"] * 100
    ds = ds.rename("mcs_precip_percentage")

    # save results
    ds.to_netcdf(
        f"{path}mcs_precip_percent_era5_{year}_MAM.nc"
    )


Processing year 1991

Processing year 1992

Processing year 1993

Processing year 1994

Processing year 1995

Processing year 1996

Processing year 1997

Processing year 1998

Processing year 1999

Processing year 2000

Processing year 2001


In [11]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/era5/blobs/"
for i in range(len(b_files)):

    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")
    year = np.unique(ds_b.time.dt.year)[0]
    print(f"\nProcessing year {year}")

    ds_b = ds_b.sel(time=ds_b.time.dt.month.isin([6, 7, 8]))
    ds_b_total = ds_b.mean(dim='time') * len(ds_b.time)

    p_files = sorted(
        p_path.glob(f"e5.oper.fc.sfc.meanflux.235_055_mtpr.ll025sc*{year}*.nc")
    )
    ds_p = (
        xr.open_mfdataset(p_files)
        .sel(latitude=slice(55, 5))
        .sel(longitude=slice(235, 295))
        .sel(time=slice(f"{str(year)}-01-01", f"{str(year)}-12-31"))
        .load()
    )
    ds_p = ds_p.rename({'latitude': 'lat', 'longitude': 'lon'})

    ds_p = ds_p.sel(time=ds_p.time.dt.month.isin([6, 7, 8]))
    # resample prect
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    ds_p_total = ds_p.mean(dim='time') * len(ds_p.time)

    # Compute mcs rain rate %
    ds = ds_b_total["__xarray_dataarray_variable__"] / ds_p_total["MTPR"] * 100
    ds = ds.rename("mcs_precip_percentage")

    # save results
    ds.to_netcdf(
        f"{path}mcs_precip_percent_era5_{year}_JJA.nc"
    )


Processing year 1991

Processing year 1992

Processing year 1993

Processing year 1994

Processing year 1995

Processing year 1996

Processing year 1997

Processing year 1998

Processing year 1999

Processing year 2000

Processing year 2001
